In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# read and cleanup meteorological data from around BAZAN
# From: https://ims.data.gov.il/ims/1   stations: Haifa-Batey Zikuk (600), Afek, Haifa Technion
bzn = pd.read_json('ims_bazan.json')
afek = pd.read_json('ims_afek.json')
technion = pd.read_json('ims_technion.json')

ims = bzn.merge(afek, on='time_obs', suffixes=('', ' Afek'))
ims = ims.merge(technion, on='time_obs', suffixes=(' Bazan', ' Technion'))

ims['dt'] = pd.to_datetime(ims['time_obs'])
ims['date'] = ims['dt'].dt.date
ims['hour'] = ims['dt'].dt.hour
ims = ims.mask(ims == -9999)

ims.dropna(axis=1, how='all', inplace=True)
#print(ims.columns)
#ims.describe()

In [ ]:
c1 = 'tmp_air_dry Bazan'
c2 = 'tmp_air_dry Technion'
ims.plot.scatter(x=c1, y=c2)
plt.show()

In [ ]:
ims.index = ims['time_obs']
# Technion is 3.7km from Bazan horizontally, and elevated +240 meters
ims['tmp delta'] = ims['tmp_air_dry Bazan'] - ims['tmp_air_dry Technion']
ims['month'] = ims['dt'].dt.month
ims['td_cat'] = pd.qcut(ims['tmp delta'], 6)



In [ ]:
#fig, axs = plt.subplots(1, 2)

table = pd.crosstab(ims['month'], [ims['td_cat']], normalize='index')
axs[0] = table.plot.bar(stacked=True)
axs[0].legend(loc='upper right', bbox_to_anchor=(1.4, 1.05),
          ncol=1, fancybox=True, shadow=True)
axs[0].set_title('Temp. delta distribution by month')

table = pd.crosstab(ims['hour'], [ims['td_cat']], normalize='index')
axs[1] = table.plot.bar(stacked=True)
axs[1].legend(loc='upper right', bbox_to_anchor=(1.4, 1.05),
          ncol=1, fancybox=True, shadow=True)
axs[1].set_title('Temp. delta distribution by hour')

plt.show()

In [ ]:
#byday = ims.groupby(['date']).mean()
byday = ims

byday['dt'] = pd.to_datetime(byday.index)
byday['month'] = byday['dt'].dt.month
byday['wind_spd Bazan'].plot()
byday['wind cat'] = pd.qcut(byday['wind_spd Bazan'], 10)

table = pd.crosstab(byday['month'], [byday['wind cat']], normalize='index')
ax = table.plot.bar(stacked=True)
ax.legend(loc='upper right', bbox_to_anchor=(1.4, 1.05),
          ncol=1, fancybox=True, shadow=True)
ax.set_title('Wind speed distribution by month')

table = pd.crosstab(byday['hour'], [byday['wind cat']], normalize='index')
ax = table.plot.bar(stacked=True)
ax.legend(loc='upper right', bbox_to_anchor=(1.4, 1.05),
          ncol=1, fancybox=True, shadow=True)
ax.set_title('Wind speed distribution by hour')

plt.show()



In [ ]:
# joint inversion index
inv_days = ims
inv_days['bywind'] = (inv_days['wind_spd Bazan'] < 1.0).astype(int)
inv_days['bytemp_delta'] = ((ims['tmp_air_dry Bazan'] - ims['tmp_air_dry Technion']) < 0.0).astype(int)
inv_days['byboth'] = (inv_days['bywind'] & inv_days['bytemp_delta'])
inv_days = inv_days.groupby(['date']).sum()

inv_idx = inv_days[(inv_days['bywind'] + inv_days['bytemp_delta']) > 0][['bywind', 'bytemp_delta', 'byboth']]
inv_idx.to_csv('inversion_index.csv')